In [2]:
import pandas as pd

In [4]:
reviewlist = pd.read_csv('define_datareview_list.csv')

In [246]:
pd.set_option('display.max_colwidth', -1)
(reviewlist)

,Unnamed: 0,datasets,method
0,deployment0001,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0001_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20140415T132159.798580-20140706T212524.736690.nc,recovered_host
1,deployment0002,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0002_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20150509T113514.756870-20150625T150702.264890.nc,recovered_host
2,deployment0003,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0003_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20160121T150043.663730-20160518T015328.144100.nc,recovered_host
3,deployment0004,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174435-CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/deployment0004_CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument_20170116T125355.760380-20170116T221950.330870.nc,telemetered


In [32]:
import functions.common as cf
import functions.plotting as pf
import xarray as xr

In [33]:
col = list(reviewlist.columns)
refdes = reviewlist[col[1]][0].split('/')[-1].split('_')[1][0:27]
dr_data = cf.refdes_datareview_json(refdes)

In [249]:
df = pd.DataFrame()
for ii in range(len(reviewlist)):         
    deploy_num = int(reviewlist[col[0]][ii].split('t')[-1])
    deploy_info = cf.get_deployment_information(dr_data, deploy_num)
    deploy_depth = deploy_info['deployment_depth']
    
    # Check deployment pressure from asset management against pressure variable in file
    ds = xr.open_dataset(reviewlist[col[1]][ii], mask_and_scale=False)
    ds = ds.swap_dims({'obs': 'time'})
    press = pf.pressure_var(ds, list(ds.coords.keys()))
    if press is None:
        press = pf.pressure_var(ds, list(ds.data_vars.keys()))     
    pressure_compare, pressure_max, pressure_mean = cf.calculate_mean_pressure(press, ds, refdes, deploy_depth)
        
    # check coordinate
    file_coordinates = list(ds.coords.keys())
    if 'SBD' not in refdes.split('-')[1]:
        check_coords = list(set(['obs', 'time', 'pressure', 'lat', 'lon']) - set(file_coordinates))
    else:
        check_coords = list(set(['obs', 'time', 'lat', 'lon']) - set(file_coordinates))
    
    if len(check_coords) > 0:
        if 'pressure' in check_coords:
            if len([j for j in file_coordinates if 'pressure' in j]) == 1:
                check_coords.remove('pressure')
                if len(check_coords) > 0:
                    coord_test = 'missing: {}'.format(check_coords)
                else:
                    coord_test = 'pass'
            else:
                coord_test = 'missing: {}'.format(check_coords)
        else:
            coord_test = 'missing: {}'.format(check_coords)
    else:
        coord_test = 'pass'
    df0 = pd.DataFrame({'pressure_comp': [[deploy_depth, pressure_max]],
                        'method': reviewlist[col[2]][ii],
                        'cord_test':[coord_test]}, index=[deploy_num])
    df = df.append(df0)

In [250]:
pd.set_option('display.max_colwidth', -1)
(df)

,pressure_comp,method,cord_test
1,"[1000, 832.95]",recovered_host,pass
2,"[1000, 986.27]",recovered_host,pass
3,"[1000, 961.27]",recovered_host,pass
4,"[1000, 261.65]",telemetered,pass


In [90]:
list(df.index)

[1, 2, 3, 4]

In [251]:
depl_loc_diff = pd.DataFrame(columns=list(df.index), index=list(df.index))
depl_loc_diff

,1,2,3,4
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [252]:
y = cf.deploy_location_check(refdes)
delpoy_y_loc = pd.DataFrame()
for ii in y:
    delpoy_y_loc = delpoy_y_loc.append(pd.DataFrame({'loc-t0':ii.split('_')[4].split('D')[1],
                                                     'loc-tn':ii.split('_')[2].split('D')[1],
                                                     'diff (km)': round(y[ii])}, index=[ii]))
delpoy_y_loc = delpoy_y_loc.sort_values(by=['loc-t0'])
delpoy_y_loc.index = delpoy_y_loc['loc-t0']
delpoy_y_loc = delpoy_y_loc[['loc-tn','diff (km)']]


xy = delpoy_y_loc.groupby(delpoy_y_loc.index)

In [253]:
for jj in list(df.index): 
    tn = xy.get_group(str(jj))
    for ii in list(df.index):
        x = tn[tn['loc-tn'] == str(ii)]
        if len(x) != 0: 
            depl_loc_diff[ii][jj] = tn['diff (km)'][tn['loc-tn'] == str(ii)].values[0]
            depl_loc_diff[jj][ii] = tn['diff (km)'][tn['loc-tn'] == str(ii)].values[0]
            print(ii, jj, len(x),depl_loc_diff[ii][jj], depl_loc_diff[jj][ii])
        else:
            depl_loc_diff[ii][jj] = '-' 
test

2 1 1 0 0
3 1 1 18 18
4 1 1 28 28
3 2 1 18 18
4 2 1 28 28
4 3 1 28 28


,1,2,3,4
1,-,0,18,28
2,-,-,18,28
3,-,-,-,28
4,-,-,-,-
